<a href="https://colab.research.google.com/github/ayushman1309/ML_LAB/blob/main/BST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Objective:
Implement Bayesian Decision Theory (BDT) for Classification

Task:
Write a Python program to classify a given dataset using Bayesian Decision Theory.
Dataset used: earthquake_data_tsunami.csv
Target variable: 'tsunami' (1 = Tsunami occurred, 0 = No tsunami)
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import math

df = pd.read_csv("earthquake_data_tsunami.csv")

# Separate features and target
X = df.drop(columns=['tsunami']).values
y = df['tsunami'].values


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

def multivariate_gaussian(x, mean, cov):
    size = len(x)
    det = np.linalg.det(cov)
    if det == 0:
        cov += 1e-6 * np.eye(len(cov))
        det = np.linalg.det(cov)
    norm_const = 1.0 / (math.pow((2 * np.pi), size / 2) * math.sqrt(det))
    x_mu = np.matrix(x - mean)
    inv = np.linalg.inv(cov)
    result = math.pow(math.e, -0.5 * (x_mu * inv * x_mu.T))
    return float(norm_const * result)

class BayesianDecisionClassifier:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.means = {}
        self.covs = {}
        self.priors = {}

        for c in self.classes:
            X_c = X[y == c]
            self.means[c] = np.mean(X_c, axis=0)
            self.covs[c] = np.cov(X_c, rowvar=False) + 1e-6 * np.eye(X.shape[1])
            self.priors[c] = len(X_c) / len(X)

    def predict(self, X):
        preds = []
        for x in X:
            probs = []
            for c in self.classes:
                likelihood = multivariate_gaussian(x, self.means[c], self.covs[c])
                posterior = likelihood * self.priors[c]
                probs.append(posterior)
            preds.append(self.classes[np.argmax(probs)])
        return np.array(preds)

model = BayesianDecisionClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=3)

print("=== Bayesian Decision Theory Classification ===")
print(f"Accuracy: {acc * 100:.2f}%\n")
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", report)



=== Bayesian Decision Theory Classification ===
Accuracy: 80.43%

Confusion Matrix:
 [[114  30]
 [ 16  75]]

Classification Report:
               precision    recall  f1-score   support

           0      0.877     0.792     0.832       144
           1      0.714     0.824     0.765        91

    accuracy                          0.804       235
   macro avg      0.796     0.808     0.799       235
weighted avg      0.814     0.804     0.806       235



/tmp/ipython-input-3624997584.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = math.pow(math.e, -0.5 * (x_mu * inv * x_mu.T))
